In [32]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt
import pandas as pd
from src.model import BasicModel

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False, matching_type="general", jump=3):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
        self.matching = matching_type
        self.jump = jump
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    ['week sin',
    'week cos'],
    ['year sin',
    'year cos'],
    'season idx',
    'weather idx',
    'avg ta',
    'avg rhm'
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
#     "multivariate (year)": [
#         'energy (kw 15min)',
#         'year sin',
#         'year cos',
#     ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
#     "multivariate (avg ta)": [
#         'energy (kw 15min)',
#         'avg ta'
#     ],
#     "multivariate (avg rhm)": [
#         'energy (kw 15min)',
#         'avg rhm'
#     ],
#     "multivariate (season)": [
#         'energy (kw 15min)',
#         'season idx'
#     ]
}

# Input Report Simulation
IRS = []

for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )
    
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch,
                   matching_type=_.matching,
                   is_multi_step=True,
                   model_type="Multi Step / SingleShot")
    
    bm.set_window(OUT_STEPS=3)
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_multi_predict(is_reshape=True)
    bm.set_multi_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (weather)) set predict (validation) info start ###### 

0 / 99
50 / 99
99 / 99 complete.

###### [Notice] set predict info success ###### 



In [34]:
test_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.865588,53.331016,0.325137,0.012961,1.909705,0.867244,0.209604,1244.984038,581.021401
multivariate (week) / with cluster,0.868161,54.153489,0.325902,0.010130,1.891109,0.877151,0.205910,1244.984038,570.781740
multivariate (weather) / with cluster,0.866806,54.234122,0.324312,0.006859,1.901955,0.866591,0.205548,1244.984038,569.777876
univariate,0.880246,58.876357,0.304577,0.016878,1.830663,0.872417,0.184698,1244.984038,511.982797
multivariate (week),0.894793,63.182978,0.291051,0.008048,1.712002,0.886289,0.165356,1244.984038,458.366043
multivariate (weather),0.880162,58.879337,0.305122,0.014207,1.831438,0.872231,0.184685,1244.984038,511.945690


In [35]:
val_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.921791,71.324847,0.233388,0.003732,1.365036,0.899514,0.095754,694.234627,199.072840
multivariate (week) / with cluster,0.923286,71.829103,0.230354,0.002943,1.344349,0.899610,0.094070,694.234627,195.572121
multivariate (weather) / with cluster,0.920699,70.936795,0.235932,0.003379,1.372614,0.897990,0.097050,694.234627,201.766836
univariate,0.913433,67.989691,0.236635,0.005435,1.419514,0.892232,0.106891,694.234627,222.226648
multivariate (week),0.915310,68.668368,0.235935,0.007944,1.382555,0.895674,0.104625,694.234627,217.515037
multivariate (weather),0.914173,68.326496,0.234153,0.003968,1.411612,0.892624,0.105766,694.234627,219.888434
